In [23]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
import json
from langchain.agents import AgentExecutor, Tool
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder,AIMessagePromptTemplate

from langchain.memory import ConversationBufferMemory

from langchain.agents import AgentExecutor, create_structured_chat_agent

In [24]:
load_dotenv()

True

In [25]:
llm = ChatOpenAI(model="gpt-4o",temperature=0.1)

In [26]:
def get_local_time(input: str = None) -> str:
    from datetime import datetime
    current_time = datetime.now().strftime("%H:%M:%S")
    # print(f"LocalTime tool called, returning: {current_time}")
    return f"Current local time is: {current_time}"

In [27]:
tools = [
    Tool(
        name="LocalTime",
        func=get_local_time,
        description="Use this tool to get the current local time."
    )
]

In [28]:
system_message=SystemMessagePromptTemplate.from_template(
    """ You are an AI assistant. Your responses will be converted into speech.
        Please ensure that your answers:
        - Are conversational and sound natural when spoken.
        - Use a friendly and engaging tone.
        - Avoid overly formal or robotic language.
        - Use contractions where appropriate (e.g., "I'm" instead of "I am").
        - Keep sentences short and easy to understand.
        - Use a storytelling style when possible.
        
        Respond to the human as helpfully and accurately as possible. You have access to the following tools:
        {tools}
        Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
        Valid "action" values: "Final Answer" or {tool_names}.
        Provide only ONE action per $JSON_BLOB, as shown:
        ```
        {{
        "action": $TOOL_NAME,
        "action_input": $INPUT
        }}
        ```
        Follow this format:

        Question: input question to answer
        Thought: consider previous and subsequent steps
        Action:
        ```
        $JSON_BLOB
        ```
        Observation: action result
        ... (repeat Thought/Action/Observation N times)
        Thought: I know what to respond
        Action:
        ```
        {{
        "action": "Final Answer",
        "action_input": "Final response to human"
        }}
        ```        
        Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation
        """
)

history_placeholder=MessagesPlaceholder(variable_name="history",optional=True)

scratchpad_placeholder = MessagesPlaceholder(variable_name="agent_scratchpad", optional=True)

human_message=HumanMessagePromptTemplate.from_template(
    """{input}
    
    {agent_scratchpad}
    
    (reminder to respond in a JSON blob no matter what)"""
)

assisstant_message=AIMessagePromptTemplate.from_template(
    "Understood! I will analyze your question type and provide a detailed response."
)

In [29]:
prompt_template=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    human_message,
    assisstant_message
])

In [30]:
memory = ConversationBufferMemory(
    memory_key="history", return_messages=True)

In [31]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True)

In [32]:
MEMORY_FILE = "memory.json"

In [33]:
print(os.getcwd())

/Users/yiding/personal_projects/ML/github_repo/littleSeven/langc_memory_store


In [34]:
# def load_memory():
#     if os.path.exists(MEMORY_FILE):
#         with open(MEMORY_FILE, "r", encoding="utf-8") as file:
#             try:
#                 data = json.load(file)
#                 if isinstance(data, list):
#                     for message in data:
#                         if message["role"] == "human":
#                             memory.chat_memory.add_user_message(message["content"])
#                         elif message["role"] == "ai":
#                             memory.chat_memory.add_ai_message(message["content"])
#             except json.JSONDecodeError as e:
#                 return f"Cannot load history. {e}"
            
def load_memory():
    import time
    if os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, "r", encoding="utf-8") as file:
            try:
                data = json.load(file)
                
                saved_timestamp = data.get("timestamp", 0)
                current_time = time.time()
                elapsed_time = current_time - saved_timestamp
                if elapsed_time > 24 * 3600: 
                    return
                
                messages = data.get("messages", [])
                for message in messages:
                    if message["role"] == "human":
                        memory.chat_memory.add_user_message(message["content"])
                    elif message["role"] == "ai":
                        memory.chat_memory.add_ai_message(message["content"])

            except json.JSONDecodeError as e:
                return f"Cannot load history. {e}"


# def save_memory():
#     if not os.path.exists(MEMORY_FILE):
#         with open(MEMORY_FILE, 'w', encoding='utf-8') as file:
#             json.dump([], file) 

#     messages_as_dicts = [{"role": msg.type, "content": msg.content} for msg in memory.chat_memory.messages]
    
#     with open(MEMORY_FILE, 'w', encoding='utf-8') as file:
#         json.dump(messages_as_dicts, file, ensure_ascii=False, indent=4)


def save_memory():
    import time
    timestamp=time.time()

    if not os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, 'w', encoding='utf-8') as file:
            json.dump([], file) 
    
    messages_as_dicts = [{"role": msg.type, "content": msg.content} for msg in memory.chat_memory.messages]

    data_to_save = {
        "timestamp": timestamp,
        "messages": messages_as_dicts
    }

    with open(MEMORY_FILE, 'w', encoding='utf-8') as file:
        json.dump(data_to_save, file, ensure_ascii=False, indent=4)


In [35]:
load_memory()

In [36]:
memory.chat_memory.messages

[HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the time now?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The current local time is 5:13 PM.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the time now?', additional_kwargs

In [37]:
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt_template)

In [38]:
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory,handle_parsing_errors=True, verbose=True)

In [39]:
response = agent_executor.invoke({
    "input": "What is 15^2?",
    "history": memory.chat_memory.messages,  
    # "agent_scratchpad": ''
})

print(response)



> Entering new AgentExecutor chain...


{
  "action": "Final Answer",
  "action_input": "15 squared is 225."
}

> Finished chain.
{'input': 'What is 15^2?', 'history': [HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the time now?', additional_kwargs={}, response_metadata={}), AIMessage(content='The current local time is 5:13 PM.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing great, thanks for asking! How abo

In [40]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the time now?', additional_kwargs={}, response_metadata={}), AIMessage(content='The current local time is 5:13 PM.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}), 

In [41]:
response = agent_executor.invoke({
    "input": "how are you?",
})

print(response)



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "I'm doing great, thanks for asking! How about you?"
}

> Finished chain.
{'input': 'how are you?', 'history': [HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the time now?', additional_kwargs={}, response_metadata={}), AIMessage(content='The current local time is 5:13 PM.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_m

In [42]:
response = agent_executor.invoke({
    "input": "what is the time now?",
})

print(response)



> Entering new AgentExecutor chain...
{
  "action": "LocalTime",
  "action_input": ""
}Current local time is: 17:14:18{
  "action": "Final Answer",
  "action_input": "The current local time is 5:14 PM."
}

> Finished chain.
{'input': 'what is the time now?', 'history': [HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the time now?', additional_kwargs={}, response_metadata={}), AIMessage(content='The current local time is 5:13 PM.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}), 

In [43]:
memory.chat_memory.messages

[HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the time now?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The current local time is 5:13 PM.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the time now?', additional_kwargs

In [44]:
save_memory()